In [1]:
import numpy as np
import pandas as pd
from pandas import read_csv
import json

import os, sys
from os.path import dirname


ROOT_DIR = '.'


In [3]:
#import data
raw_data = read_csv(os.path.join(ROOT_DIR,'data','data.csv'))
data = raw_data.copy()

# data rejections
reject = read_csv(os.path.join(ROOT_DIR,'data','reject.csv'))
data = data[data.subject.isin(reject.query('reject==0').subject)]

## Sort data.
f = lambda x: x.subject + '_' + x.rune
data['stimulus'] = np.unique(data.apply(f, 1), return_inverse=True)[-1] + 1
data = data.sort_values(['stimulus','exposure']).reset_index(drop=True)

data['outcome'] = data.outcome.replace({10:1, 1:0, -1:1, -10:0})
data['valence'] = data.valence.replace({'win':1, 'lose':0})



In [4]:
for i in [1,2,3]:
    data_i = data[data.session == i]
    ## Define metadata.
    N = data_i.stimulus.nunique()
    E = data_i.exposure.nunique()
    S = np.unique(data_i[data_i.exposure==1].subject, return_inverse=True)[-1] + 1

    ## Prepare task variables.
    R = data_i.pivot_table('outcome','exposure','stimulus').fillna(0).values

    ## Prepare response variables.
    Y = data_i.pivot_table('choice','exposure','stimulus').fillna(0).values.astype(int)

    ## Prepare valance variables.

    V = data_i.pivot_table('valence','exposure','stimulus').fillna(0).values.astype(int)
    
    C = data_i.pivot_table('choice','exposure','stimulus').notnull().values.astype(int)
    C = np.where(data_i.pivot_table('rt','exposure','stimulus',dropna=False).values < 0.2, 0, C)

    #print sanity check
    print(E,N)
    print(R.shape)
    print(Y.shape)
    print(V.shape)
    print(C.shape)

    model_data = dict(N=N, 
                  E=E, 
                  C=C.tolist(),
                  S=S.tolist(), 
                  Y=Y.tolist(), 
                  V=V.tolist(),
                  R=R.tolist())

#     json_data = json.dumps(model_data)

#     f = open("stan_data/s{}_data.json".format(i),"w")
#     f.write(json_data)
#     f.close()
    
#     data_i.to_csv("stan_data/s{}_dataframe.csv".format(i), index=False)

/usr/people/gk9567/.local/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less


12 2640
(12, 2640)
(12, 2640)
(12, 2640)
(12, 2640)
12 2328
(12, 2328)
(12, 2328)
(12, 2328)
(12, 2328)
12 2376
(12, 2376)
(12, 2376)
(12, 2376)
(12, 2376)


In [75]:
# # Opening JSON file
# r = open('stan_data/m1_s1_data.json')
  
# # a dictionary
# dd = json.load(r)

# #reformat
# dd['S'] = np.array(dd['S'])
# dd['Y'] = np.array(dd['Y'])
# dd['R'] = np.array(dd['R'])
# dd['V'] = np.array(dd['V'])


False